In [ ]:
import h5py
import glob
import numpy as np
import tifffile as tiff
import pandas as pd
import os
import matplotlib.pyplot as plt
from random import shuffle

In [ ]:
# Utils.py
def load_PNP_file(path):
    if os.path.exists(path) and (os.stat(path).st_size != 0):
        pnps = np.genfromtxt(path, delimiter=',')
        if pnps.ndim == 1:
            if pnps.size == 4:
                tmp = pnps.copy()
                pnps = np.empty((1, 4))
                pnps[0, :] = tmp
            else:
                pnps = np.empty((0, 4))
    else:
        pnps = np.empty([0, 4])
    return pnps

In [ ]:
imgs = np.zeros((7049, 618, 824, 3), dtype=np.int16)
pnps = []
paths = []

In [ ]:
im_nms = glob.glob("deep_learning/data/cell_shooting_new/" + '*_PNPs.csv')
shuffle(im_nms)
n_cells = 0
for i, nm in enumerate(im_nms):
    nm_i = nm.replace("_PNPs.csv", "_image.tif",)
    if os.path.exists(nm) and os.path.exists(nm_i):
        img = tiff.imread(nm_i)
        pnp = load_PNP_file(nm)
        nm_img = nm_i.replace("/mnt/data/PNP/deep_learning/data/cell_shooting_new/", "")
        imgs[n_cells] = img
        pnps.append(pnp)
        paths.append(nm_img)
        if n_cells%500 == 0:
            print(n_cells)
        n_cells += 1
            
print(n_cells)

In [ ]:
nm_i

In [ ]:
if True:
    import h5py
    f = h5py.File('deep_learning/data/cell_shooting_new/db_for_training_cells_shuffle.h5', 'w')
    dset = f.create_dataset("imgs", data=imgs)
    f.close()

In [ ]:
if True:
    ll = []
    for i, nm, p in zip(range(0, len(pnps)), paths, pnps):
        ll.append([i, nm, p])
    pnpsdf = pd.DataFrame.from_records(ll, columns=['idx', 'cell', 'pnps'])
    pnpsdf.to_hdf('deep_learning/data/cell_shooting_new/db_for_training_cells_shuffle.h5', 'a')